In [1]:
pip install silx

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.9 MB/s  0:00:006m0:00:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Simple Code to stack edf files into one hdf5 file

In [11]:
import subprocess
import glob

my_edf_scan = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/DEPow_0.000000_rockinglayer_10x_00/'

# Find all matching EDF files
pattern = f"{my_edf_scan}DEPow_0.000000_rockinglayer_10x_00*.edf"
edf_files = glob.glob(pattern)

print(f"Found {len(edf_files)} files")

# Run conversion
if edf_files:
    cmd = ['silx', 'convert'] + edf_files + ['-o', 'converted.h5']
    subprocess.run(cmd)
else:
    print("No files found!")

Found 50 files


ERROR:silx.app.convert:Output file converted.h5 exists and mode is 'w-' (default). Aborting. To append data to an existing file, use 'a' or 'r+'.


## alternative code 

In [16]:
import subprocess
import glob

my_edf_scan = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/DEPow_0.000000_rockinglayer_10x_00/'

# Find all matching EDF files
pattern = f"{my_edf_scan}DEPow_0.000000_rockinglayer_10x_00*.edf"
edf_files = glob.glob(pattern)

print(f"Found {len(edf_files)} files")

# Run conversion with overwrite mode
if edf_files:
    cmd = ['silx', 'convert'] + edf_files + ['-o', 'converted.h5', '--mode', 'w']
    subprocess.run(cmd)

Found 50 files


In [14]:
import subprocess
import glob

my_edf_scan = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/DEPow_0.000000_rockinglayer_10x_00/'

# Find all matching EDF files
pattern = f"{my_edf_scan}DEPow_0.000000_rockinglayer_10x_00*.edf"
edf_files = glob.glob(pattern)

print(f"Found {len(edf_files)} files")

# Show first few files as verification
if edf_files:
    print("First 3 files:")
    for f in edf_files[:3]:
        print(f"  {f}")

# Run conversion with overwrite mode
if edf_files:
    cmd = ['silx', 'convert'] + edf_files + ['-o', 'converted.h5', '--mode', 'w']
    
    print(f"\nRunning command with {len(edf_files)} files...")
    print(f"Output will be: converted.h5")
    
    # Capture output and errors
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Print stdout and stderr
    if result.stdout:
        print("STDOUT:")
        print(result.stdout)
    if result.stderr:
        print("STDERR:")
        print(result.stderr)
    
    # Check return code
    if result.returncode == 0:
        print("\n✓ Conversion completed successfully!")
        # Verify the file was created
        import os
        if os.path.exists('converted.h5'):
            size = os.path.getsize('converted.h5')
            print(f"  File size: {size / (1024**2):.2f} MB")
    else:
        print(f"\n✗ Conversion failed with return code: {result.returncode}")
else:
    print("No files found to convert!")

Found 50 files
First 3 files:
  /scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/DEPow_0.000000_rockinglayer_10x_00/DEPow_0.000000_rockinglayer_10x_00_pcoedge_0001_0000_0042.edf
  /scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/DEPow_0.000000_rockinglayer_10x_00/DEPow_0.000000_rockinglayer_10x_00_pcoedge_0001_0000_0035.edf
  /scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/DEPow_0.000000_rockinglayer_10x_00/DEPow_0.000000_rockinglayer_10x_00_pcoedge_0001_0000_0047.edf

Running command with 50 files...
Output will be: converted.h5

✓ Conversion completed successfully!
  File size: 527.51 MB


## Code to run all folders with edf file

In [17]:
import subprocess
import glob
import os
import re

base_dir = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/'

# Find all subdirectories
all_items = os.listdir(base_dir)
folders = []

for item in all_items:
    full_path = os.path.join(base_dir, item)
    # Check if it's a directory and matches pattern
    if os.path.isdir(full_path) and item.startswith('DEPow_0.000000_rockinglayer_10x_'):
        folders.append(full_path)

print(f"Found {len(folders)} matching folders:\n")
for folder in sorted(folders):
    print(f"  - {os.path.basename(folder)}")
print()

# Process each folder
for folder in sorted(folders):
    folder_name = os.path.basename(folder)
    print(f"Processing: {folder_name}")
    
    # Find EDF files
    edf_pattern = os.path.join(folder, '*.edf')
    edf_files = glob.glob(edf_pattern)
    
    print(f"  Found {len(edf_files)} EDF files")
    
    if edf_files:
        output_file = os.path.join(folder, f'{folder_name}_converted.h5')
        cmd = ['silx', 'convert'] + edf_files + ['-o', output_file, '--mode', 'w']
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            size = os.path.getsize(output_file)
            print(f"  ✓ Success! {size / (1024**2):.2f} MB\n")
        else:
            print(f"  ✗ Failed\n")
    else:
        print(f"  ⊘ Skipped (no EDF files)\n")

print("Done!")

Found 21 matching folders:

  - DEPow_0.000000_rockinglayer_10x_00
  - DEPow_0.000000_rockinglayer_10x_01
  - DEPow_0.000000_rockinglayer_10x_02
  - DEPow_0.000000_rockinglayer_10x_03
  - DEPow_0.000000_rockinglayer_10x_04
  - DEPow_0.000000_rockinglayer_10x_05
  - DEPow_0.000000_rockinglayer_10x_06
  - DEPow_0.000000_rockinglayer_10x_07
  - DEPow_0.000000_rockinglayer_10x_08
  - DEPow_0.000000_rockinglayer_10x_09
  - DEPow_0.000000_rockinglayer_10x_10
  - DEPow_0.000000_rockinglayer_10x_11
  - DEPow_0.000000_rockinglayer_10x_12
  - DEPow_0.000000_rockinglayer_10x_13
  - DEPow_0.000000_rockinglayer_10x_14
  - DEPow_0.000000_rockinglayer_10x_15
  - DEPow_0.000000_rockinglayer_10x_16
  - DEPow_0.000000_rockinglayer_10x_17
  - DEPow_0.000000_rockinglayer_10x_18
  - DEPow_0.000000_rockinglayer_10x_19
  - DEPow_0.000000_rockinglayer_10x_20

Processing: DEPow_0.000000_rockinglayer_10x_00
  Found 50 EDF files
  ✓ Success! 527.51 MB

Processing: DEPow_0.000000_rockinglayer_10x_01
  Found 50 ED

In [ ]:
import subprocess
import glob
import os
import re

base_dir = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_ff_morningafter/Morning_after_extracted/'

# Find all subdirectories
all_items = os.listdir(base_dir)
folders = []

for item in all_items:
    full_path = os.path.join(base_dir, item)
    # Check if it's a directory and matches pattern
    if os.path.isdir(full_path) and item.startswith('DEPow_0.000000_rockinglayer_10x_'):
        folders.append(full_path)

print(f"Found {len(folders)} matching folders:\n")
for folder in sorted(folders):
    print(f"  - {os.path.basename(folder)}")
print()

# Process each folder
for folder in sorted(folders):
    folder_name = os.path.basename(folder)
    print(f"Processing: {folder_name}")
    
    # Find EDF files
    edf_pattern = os.path.join(folder, '*.edf')
    edf_files = glob.glob(edf_pattern)
    
    print(f"  Found {len(edf_files)} EDF files")
    
    if edf_files:
        output_file = os.path.join(folder, f'{folder_name}_converted.h5')
        cmd = ['silx', 'convert'] + edf_files + ['-o', output_file, '--mode', 'w']
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            size = os.path.getsize(output_file)
            print(f"  ✓ Success! {size / (1024**2):.2f} MB\n")
        else:
            print(f"  ✗ Failed\n")
    else:
        print(f"  ⊘ Skipped (no EDF files)\n")

print("Done!")

In [15]:
import os

# Check current working directory
print(f"Current working directory: {os.getcwd()}")

# Full path to the converted file
converted_path = os.path.join(os.getcwd(), 'converted.h5')
print(f"Converted file is at: {converted_path}")

# Verify it exists
if os.path.exists(converted_path):
    print("✓ File found!")
else:
    print("✗ File not found")

Current working directory: /home/users/edemdh
Converted file is at: /home/users/edemdh/converted.h5
✓ File found!
